# 01. Criteo 데이터셋 탐색 (EDA)

**목표**:
- Criteo 데이터 구조 이해
- 기본 통계 분석
- 결측치 및 분포 확인
- 전처리 전략 수립

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ 라이브러리 임포트 완료")

## 1. 데이터 로딩 (샘플링)

전체 45M 행은 메모리에 로딩하기 어려우므로 100만 행 샘플링

In [ ]:
# 컬럼명 정의
num_features = [f'I{i}' for i in range(1, 14)]  # 13개 수치형
cat_features = [f'C{i}' for i in range(1, 27)]  # 26개 범주형
columns = ['label'] + num_features + cat_features

print(f"총 {len(columns)}개 컬럼: 1 label + 13 numeric + 26 categorical")
print(f"수치형 특징: {num_features[:3]}...")
print(f"범주형 특징: {cat_features[:3]}...")

In [ ]:
# 샘플 데이터 로딩 (1M rows)
sample_size = 1_000_000
data_path = '../data/raw/dac/train.txt'

print(f"📥 {sample_size:,}개 샘플 로딩 중...")
df = pd.read_csv(
    data_path, 
    sep='\t', 
    header=None, 
    names=columns,
    nrows=sample_size
)

print(f"✅ 데이터 로딩 완료: {df.shape}")
print(f"메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 2. 기본 정보

In [ ]:
# 데이터 미리보기
df.head(10)

In [ ]:
# 데이터 타입 및 결측치
df.info()

In [ ]:
# 라벨 분포 (CTR)
label_counts = df['label'].value_counts()
ctr = df['label'].mean()

print(f"\n📊 라벨 분포:")
print(label_counts)
print(f"\n⚡ CTR (Click-Through Rate): {ctr:.4%}")

# 시각화
plt.figure(figsize=(8, 5))
label_counts.plot(kind='bar', color=['#ff6b6b', '#4ecdc4'])
plt.title(f'Label Distribution (CTR: {ctr:.4%})', fontsize=14, fontweight='bold')
plt.xlabel('Label (0: No Click, 1: Click)')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 3. 수치형 특징 분석

In [ ]:
# 수치형 특징 통계
df[num_features].describe()

In [ ]:
# 결측치 비율
missing_pct = df[num_features].isnull().mean() * 100
missing_pct = missing_pct.sort_values(ascending=False)

print("\n📊 수치형 특징 결측치 비율:")
print(missing_pct)

# 시각화
plt.figure(figsize=(12, 5))
missing_pct.plot(kind='bar', color='coral')
plt.title('Missing Value Percentage (Numeric Features)', fontsize=14, fontweight='bold')
plt.xlabel('Features')
plt.ylabel('Missing %')
plt.axhline(y=50, color='red', linestyle='--', label='50% threshold')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 수치형 특징 분포 (첫 6개)
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for idx, col in enumerate(num_features[:6]):
    df[col].hist(bins=50, ax=axes[idx], color='skyblue', edgecolor='black')
    axes[idx].set_title(f'{col} Distribution', fontweight='bold')
    axes[idx].set_xlabel('Value')
    axes[idx].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print("\n💡 관찰: 대부분 오른쪽으로 치우친 분포 (log transformation 필요)")

## 4. 범주형 특징 분석

In [ ]:
# 범주형 특징 고유값 개수 (Cardinality)
cardinality = df[cat_features].nunique().sort_values(ascending=False)

print("\n📊 범주형 특징 Cardinality (고유값 개수):")
print(cardinality)

# 시각화
plt.figure(figsize=(14, 6))
cardinality.plot(kind='bar', color='mediumpurple')
plt.title('Cardinality of Categorical Features', fontsize=14, fontweight='bold')
plt.xlabel('Features')
plt.ylabel('Number of Unique Values')
plt.yscale('log')
plt.tight_layout()
plt.show()

print(f"\n💡 최대 cardinality: {cardinality.max():,} (매우 높음 → Hash/Frequency encoding 필요)")

In [ ]:
# 범주형 특징 결측치
cat_missing_pct = df[cat_features].isnull().mean() * 100
cat_missing_pct = cat_missing_pct.sort_values(ascending=False)

print("\n📊 범주형 특징 결측치 비율:")
print(cat_missing_pct.head(10))

# 시각화
plt.figure(figsize=(14, 6))
cat_missing_pct.plot(kind='bar', color='lightcoral')
plt.title('Missing Value Percentage (Categorical Features)', fontsize=14, fontweight='bold')
plt.xlabel('Features')
plt.ylabel('Missing %')
plt.tight_layout()
plt.show()

## 5. 상관 관계 분석

In [ ]:
# 수치형 특징 간 상관관계
corr_matrix = df[num_features].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=0.5)
plt.title('Correlation Matrix (Numeric Features)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 라벨과의 상관관계
label_corr = df[num_features + ['label']].corr()['label'].drop('label').sort_values(ascending=False)

print("\n📊 라벨과의 상관관계 (수치형 특징):")
print(label_corr)

plt.figure(figsize=(10, 6))
label_corr.plot(kind='barh', color='teal')
plt.title('Correlation with Label', fontsize=14, fontweight='bold')
plt.xlabel('Correlation Coefficient')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()

## 6. 전처리 전략 수립

In [ ]:
print("="*60)
print("📋 전처리 전략 요약")
print("="*60)

print("\n1️⃣ 수치형 특징 (13개):")
print("   - 결측치 처리: 중앙값(median) 대체")
print("   - 로그 변환: np.log1p() 적용 (오른쪽 꼬리 분포 완화)")
print("   - 정규화: MinMaxScaler 또는 StandardScaler")

print("\n2️⃣ 범주형 특징 (26개):")
print("   - 결측치 처리: 'missing' 카테고리 생성")
print("   - High-cardinality 처리:")
print("     → 빈도 기반 필터링 (threshold=10)")
print("     → Hash encoding 또는 Target encoding")
print("   - 최종 인코딩: Label Encoding → Embedding Layer")

print("\n3️⃣ 데이터 분할:")
print("   - Train: 80% (36M rows)")
print("   - Validation: 10% (4.5M rows)")
print("   - Test: 10% (4.5M rows)")

print("\n4️⃣ 시퀀스 생성 (Taobao용):")
print("   - Criteo는 정적 특징 위주 → DCNv3에 유리")
print("   - Taobao는 순차 데이터 → Mamba4Rec에 유리")

print("\n5️⃣ 샘플링 전략:")
print("   - 초기 실험: 1-2M rows (빠른 반복)")
print("   - 최종 실험: 전체 45M rows (클라우드 GPU)")

print("\n" + "="*60)

## 7. 다음 단계

✅ **완료**: 데이터 구조 이해 및 EDA

**TODO**:
1. `experiments/preprocessing.py` 작성
2. 전처리 파이프라인 구현
3. 처리된 데이터 저장 (.parquet)